# Credit Card Fraud Detection using Machine Learning

## Project Overview

This notebook develops a machine learning model to detect and prevent credit card fraud by identifying suspicious transactions. Machine learning provides an effective approach to automatically identify fraudulent transactions based on patterns in the data.

### What is Anomaly Detection?

**Anomaly detection** (also called outlier detection) is a technique used to identify rare items, events, or observations that deviate significantly from the majority of the data. In the context of credit card fraud:

- **Normal transactions** represent the majority of data (legitimate purchases)
- **Anomalies/Outliers** represent fraudulent transactions that exhibit unusual patterns

There are two main approaches to anomaly detection:
1. **Supervised Learning**: When we have labeled data (fraud/not fraud), we can train classifiers
2. **Unsupervised Learning**: When labels are unavailable, we identify anomalies based on statistical properties

Since our dataset is **labeled** (has a `Class` column), we'll use **supervised classification** while also exploring some unsupervised methods for comparison.

### Dataset Features

- `id`: Unique identifier for each transaction
- `V1-V28`: Anonymized features (result of PCA transformation for privacy)
- `Amount`: Transaction amount
- `Class`: Target variable (0 = legitimate, 1 = fraudulent)

## 1. Setup and Imports

We'll import all necessary libraries for data manipulation, visualization, and machine learning.

In [ ]:
# Data manipulation and numerical operations
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

# Machine Learning Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

# Metrics and Evaluation
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    precision_score, 
    recall_score, 
    f1_score,
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
    average_precision_score
)

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Display settings
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

print("All libraries imported successfully!")

## 2. Data Loading and Initial Exploration

Let's load the dataset and perform initial exploration to understand its structure.

In [ ]:
# Load the dataset
# IMPORTANT: Update this path to your actual dataset location
df = pd.read_csv('creditcard_2023.csv')

# Display basic information
print("="*60)
print("DATASET OVERVIEW")
print("="*60)
print(f"\nDataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nMemory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Display first few rows
print("\nFirst 5 rows of the dataset:")
df.head()

In [ ]:
# Check data types and non-null counts
print("\nData Types and Non-Null Counts:")
print("-"*40)
df.info()

In [ ]:
# Statistical summary
print("\nStatistical Summary:")
df.describe()

## 3. Data Quality Assessment

Before building our model, we need to check for:
- Missing values
- Duplicate records
- Data types consistency

In [ ]:
print("="*60)
print("DATA QUALITY ASSESSMENT")
print("="*60)

# Check for missing values
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

print("\n1. MISSING VALUES ANALYSIS")
print("-"*40)
if missing_values.sum() == 0:
    print(" No missing values found in the dataset!")
else:
    print("Missing values found:")
    missing_df = pd.DataFrame({
        'Missing Count': missing_values[missing_values > 0],
        'Percentage': missing_percentage[missing_values > 0]
    })
    print(missing_df)

# Check for duplicates
print("\n2. DUPLICATE RECORDS")
print("-"*40)
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates:,}")
print(f"Percentage of duplicates: {(duplicates/len(df))*100:.2f}%")

# Check unique IDs
print("\n3. ID UNIQUENESS")
print("-"*40)
unique_ids = df['id'].nunique()
print(f"Total records: {len(df):,}")
print(f"Unique IDs: {unique_ids:,}")
print(f"IDs are unique: {unique_ids == len(df)}")

## 4. Class Distribution Analysis (CRITICAL for Fraud Detection)

### Why Class Balance Matters

In fraud detection, class imbalance is a critical concern:

- **Imbalanced datasets** (typical in real-world fraud): ~99% legitimate, ~1% fraudulent
- **Balanced datasets**: Roughly equal distribution between classes

**Impact of Imbalance:**
- Models may become biased toward the majority class
- High accuracy can be misleading (predicting all as legitimate gives 99% accuracy but catches 0 frauds)
- Special techniques needed: oversampling, undersampling, cost-sensitive learning

Let's analyze our dataset's class distribution:

In [ ]:
print("="*60)
print("CLASS DISTRIBUTION ANALYSIS")
print("="*60)

# Calculate class distribution
class_counts = df['Class'].value_counts()
class_percentages = df['Class'].value_counts(normalize=True) * 100

print("\nClass Distribution:")
print("-"*40)
print(f"Legitimate Transactions (Class 0): {class_counts[0]:,} ({class_percentages[0]:.2f}%)")
print(f"Fraudulent Transactions (Class 1): {class_counts[1]:,} ({class_percentages[1]:.2f}%)")
print(f"\nClass Ratio (Legitimate:Fraudulent): {class_counts[0]/class_counts[1]:.2f}:1")

# Determine if balanced
imbalance_ratio = class_counts[0] / class_counts[1]
if 0.8 <= class_percentages[1]/class_percentages[0] <= 1.2:
    balance_status = "BALANCED"
    balance_note = """The dataset is balanced, meaning both classes have similar representation.
This simplifies our modeling approach as we don't need to apply special techniques
like SMOTE (Synthetic Minority Over-sampling Technique) or class weights."""
else:
    balance_status = "IMBALANCED"
    balance_note = "The dataset is imbalanced. Consider using techniques like SMOTE or class weights."

print(f"\nDataset Status: {balance_status}")
print(f"\nNote: {balance_note}")

In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
colors = ['#2ecc71', '#e74c3c']
bars = axes[0].bar(['Legitimate (0)', 'Fraudulent (1)'], class_counts.values, color=colors, edgecolor='black')
axes[0].set_title('Class Distribution (Count)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Number of Transactions', fontsize=12)
axes[0].set_xlabel('Transaction Class', fontsize=12)

# Add count labels on bars
for bar, count in zip(bars, class_counts.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1000, 
                 f'{count:,}', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Pie chart
axes[1].pie(class_counts.values, labels=['Legitimate', 'Fraudulent'], 
            autopct='%1.1f%%', colors=colors, explode=(0, 0.05),
            shadow=True, startangle=90)
axes[1].set_title('Class Distribution (Percentage)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('class_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Feature Analysis

### Understanding the Features

The features V1-V28 are the result of **Principal Component Analysis (PCA)** transformation. This was done to:
1. **Protect sensitive information** (anonymization)
2. **Reduce dimensionality** while preserving variance
3. **Remove multicollinearity** between original features

The `Amount` feature was NOT transformed and represents the actual transaction amount.

In [ ]:
# Analyze the Amount feature
print("="*60)
print("AMOUNT FEATURE ANALYSIS")
print("="*60)

print("\nAmount Statistics by Class:")
print("-"*40)
amount_stats = df.groupby('Class')['Amount'].describe()
print(amount_stats)

# Compare mean and median
print("\n\nKey Observations:")
print("-"*40)
legit_mean = df[df['Class']==0]['Amount'].mean()
fraud_mean = df[df['Class']==1]['Amount'].mean()
legit_median = df[df['Class']==0]['Amount'].median()
fraud_median = df[df['Class']==1]['Amount'].median()

print(f"Legitimate - Mean: ${legit_mean:.2f}, Median: ${legit_median:.2f}")
print(f"Fraudulent - Mean: ${fraud_mean:.2f}, Median: ${fraud_median:.2f}")

In [ ]:
# Visualize Amount distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution by class
for cls, color, label in [(0, '#2ecc71', 'Legitimate'), (1, '#e74c3c', 'Fraudulent')]:
    subset = df[df['Class'] == cls]['Amount']
    axes[0].hist(subset, bins=50, alpha=0.6, color=color, label=label, density=True)

axes[0].set_title('Transaction Amount Distribution by Class', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Amount ($)', fontsize=12)
axes[0].set_ylabel('Density', fontsize=12)
axes[0].legend()
axes[0].set_xlim(0, df['Amount'].quantile(0.99))  # Limit to 99th percentile for better visualization

# Box plot
df.boxplot(column='Amount', by='Class', ax=axes[1])
axes[1].set_title('Amount Box Plot by Class', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Class (0=Legitimate, 1=Fraudulent)', fontsize=12)
axes[1].set_ylabel('Amount ($)', fontsize=12)
plt.suptitle('')  # Remove automatic title

plt.tight_layout()
plt.savefig('amount_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Analyze V features - Distribution comparison
v_features = [f'V{i}' for i in range(1, 29)]

# Calculate mean differences between classes
mean_diff = []
for feature in v_features:
    fraud_mean = df[df['Class']==1][feature].mean()
    legit_mean = df[df['Class']==0][feature].mean()
    mean_diff.append(abs(fraud_mean - legit_mean))

# Sort features by discriminative power
feature_importance_df = pd.DataFrame({
    'Feature': v_features,
    'Mean_Difference': mean_diff
}).sort_values('Mean_Difference', ascending=False)

print("="*60)
print("TOP DISCRIMINATIVE FEATURES (by mean difference)")
print("="*60)
print("\nFeatures with largest mean difference between classes:")
print("(These features may be most useful for distinguishing fraud)")
print("-"*40)
print(feature_importance_df.head(10).to_string(index=False))

In [ ]:
# Visualize top discriminative features
top_features = feature_importance_df['Feature'].head(6).tolist()

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, feature in enumerate(top_features):
    for cls, color, label in [(0, '#2ecc71', 'Legitimate'), (1, '#e74c3c', 'Fraudulent')]:
        subset = df[df['Class'] == cls][feature]
        axes[idx].hist(subset, bins=50, alpha=0.6, color=color, label=label, density=True)
    
    axes[idx].set_title(f'{feature} Distribution', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel('Density')
    axes[idx].legend()

plt.suptitle('Top 6 Most Discriminative Features', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('top_features_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Correlation Analysis

In [ ]:
# Calculate correlations with target
correlations = df.drop('id', axis=1).corr()['Class'].drop('Class').sort_values(key=abs, ascending=False)

print("="*60)
print("CORRELATION WITH TARGET (Class)")
print("="*60)
print("\nTop 10 features most correlated with fraud:")
print("-"*40)
print(correlations.head(10))

print("\n\nBottom 10 features least correlated with fraud:")
print("-"*40)
print(correlations.tail(10))

In [ ]:
# Visualize correlations with target
plt.figure(figsize=(12, 6))

colors = ['#e74c3c' if x < 0 else '#2ecc71' for x in correlations.values]
plt.barh(range(len(correlations)), correlations.values, color=colors)
plt.yticks(range(len(correlations)), correlations.index)
plt.xlabel('Correlation Coefficient', fontsize=12)
plt.title('Feature Correlations with Fraud (Class)', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.savefig('correlation_with_target.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Data Preprocessing

### Preprocessing Steps:

1. **Remove non-predictive features**: The `id` column is just an identifier
2. **Feature Scaling**: Important for algorithms like Logistic Regression
   - V1-V28 are already scaled (from PCA)
   - `Amount` needs scaling

### Why Scale Features?

- **Gradient-based algorithms** (Logistic Regression, Neural Networks) converge faster
- **Distance-based algorithms** (KNN, SVM) are sensitive to feature magnitudes
- **Tree-based algorithms** (Random Forest, XGBoost) are scale-invariant but scaling doesn't hurt

In [ ]:
print("="*60)
print("DATA PREPROCESSING")
print("="*60)

# Create a copy for preprocessing
df_processed = df.copy()

# Remove the id column
print("\n1. Removing non-predictive features...")
print("    Dropping 'id' column")
df_processed = df_processed.drop('id', axis=1)

# Separate features and target
X = df_processed.drop('Class', axis=1)
y = df_processed['Class']

print(f"\n2. Feature matrix shape: {X.shape}")
print(f"   Target vector shape: {y.shape}")

# Scale the Amount feature using RobustScaler
# RobustScaler is better for data with outliers as it uses median and IQR
print("\n3. Scaling 'Amount' feature using RobustScaler...")
print("    RobustScaler chosen because Amount has outliers")
print("    Uses median and IQR instead of mean and std")

scaler = RobustScaler()
X['Amount'] = scaler.fit_transform(X[['Amount']])

print("\n Preprocessing complete!")
print(f"\nFinal feature statistics:")
print(X.describe().T[['mean', 'std', 'min', 'max']].head(10))

## 8. Train-Test Split

### Why Split the Data?

We need to evaluate our model on data it has never seen during training. This gives us an unbiased estimate of real-world performance.

### Split Strategy:

- **Training Set (80%)**: Used to train the model
- **Test Set (20%)**: Used for final evaluation (held out, never used during training)

### Important: Stratified Splitting

We use **stratified splitting** to ensure both training and test sets have the same class distribution as the original dataset. This is crucial for classification problems to ensure representative evaluation.

In [ ]:
print("="*60)
print("TRAIN-TEST SPLIT")
print("="*60)

# Perform stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.20,           # 20% for testing
    random_state=RANDOM_STATE, # For reproducibility
    stratify=y                 # Maintain class distribution
)

print("\nSplit Configuration:")
print("-"*40)
print(f" Test size: 20%")
print(f" Stratified: Yes (maintains class proportions)")
print(f" Random state: {RANDOM_STATE} (for reproducibility)")

print("\n\nDataset Sizes:")
print("-"*40)
print(f" Training set: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f" Test set: {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

# Verify stratification
print("\n\nClass Distribution Verification:")
print("-"*40)
print(f"Original - Class 0: {(y==0).sum()/len(y)*100:.2f}%, Class 1: {(y==1).sum()/len(y)*100:.2f}%")
print(f"Training - Class 0: {(y_train==0).sum()/len(y_train)*100:.2f}%, Class 1: {(y_train==1).sum()/len(y_train)*100:.2f}%")
print(f"Test     - Class 0: {(y_test==0).sum()/len(y_test)*100:.2f}%, Class 1: {(y_test==1).sum()/len(y_test)*100:.2f}%")

print("\n Stratification maintained class proportions in both sets!")

## 9. Model Selection and Training

### Models to Evaluate:

1. **Logistic Regression**: Simple, interpretable baseline
2. **Decision Tree**: Non-linear, easy to interpret
3. **Random Forest**: Ensemble of trees, robust to overfitting
4. **XGBoost**: Gradient boosting, often best performance

### Why These Models?

| Model | Pros | Cons |
|-------|------|------|
| Logistic Regression | Fast, interpretable, good baseline | Limited to linear relationships |
| Decision Tree | Handles non-linearity, interpretable | Prone to overfitting |
| Random Forest | Reduces overfitting, handles complex patterns | Less interpretable, slower |
| XGBoost | State-of-the-art performance, handles imbalance | Complex to tune, less interpretable |

In [ ]:
print("="*60)
print("MODEL TRAINING")
print("="*60)

# Define models
models = {
    'Logistic Regression': LogisticRegression(
        random_state=RANDOM_STATE,
        max_iter=1000,
        solver='lbfgs'
    ),
    'Decision Tree': DecisionTreeClassifier(
        random_state=RANDOM_STATE,
        max_depth=10,  # Limit depth to prevent overfitting
        min_samples_split=10
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=100,
        random_state=RANDOM_STATE,
        max_depth=15,
        min_samples_split=10,
        n_jobs=-1  # Use all CPU cores
    ),
    'XGBoost': XGBClassifier(
        n_estimators=100,
        random_state=RANDOM_STATE,
        max_depth=6,
        learning_rate=0.1,
        use_label_encoder=False,
        eval_metric='logloss'
    )
}

# Train all models and store results
trained_models = {}
training_results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train the model
    model.fit(X_train, y_train)
    trained_models[name] = model
    
    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Store results
    training_results[name] = {
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_pred_proba)
    }
    
    print(f"    {name} trained successfully!")

print("\n" + "="*60)
print("All models trained successfully!")
print("="*60)

## 10. Model Evaluation Metrics

### Understanding the Metrics

For fraud detection, **accuracy alone is NOT sufficient**. Here's why each metric matters:

| Metric | Formula | What it Measures | Why it Matters for Fraud |
|--------|---------|------------------|-------------------------|
| **Accuracy** | (TP+TN)/(Total) | Overall correctness | Can be misleading with imbalanced data |
| **Precision** | TP/(TP+FP) | Of predicted frauds, how many are real? | High FP = legitimate transactions blocked |
| **Recall** | TP/(TP+FN) | Of actual frauds, how many caught? | High FN = frauds go undetected (costly!) |
| **F1 Score** | 2×(P×R)/(P+R) | Balance of Precision & Recall | Overall fraud detection effectiveness |
| **ROC-AUC** | Area under ROC curve | Ability to distinguish classes | Performance across all thresholds |

### Key Concepts:

- **True Positive (TP)**: Correctly identified fraud
- **True Negative (TN)**: Correctly identified legitimate transaction
- **False Positive (FP)**: Legitimate transaction flagged as fraud (customer inconvenience)
- **False Negative (FN)**: Fraud not detected (financial loss!)

### For Fraud Detection:
- **Recall is often most important** because missing a fraud (FN) is typically more costly than a false alarm (FP)
- However, very low precision creates too many false alarms, frustrating legitimate customers

In [ ]:
print("="*60)
print("MODEL COMPARISON - EVALUATION METRICS")
print("="*60)

# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Model': list(training_results.keys()),
    'Accuracy': [training_results[m]['accuracy'] for m in training_results],
    'Precision': [training_results[m]['precision'] for m in training_results],
    'Recall': [training_results[m]['recall'] for m in training_results],
    'F1 Score': [training_results[m]['f1'] for m in training_results],
    'ROC-AUC': [training_results[m]['roc_auc'] for m in training_results]
})

# Format as percentages
comparison_display = comparison_df.copy()
for col in ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC-AUC']:
    comparison_display[col] = comparison_display[col].apply(lambda x: f"{x*100:.2f}%")

print("\n")
print(comparison_display.to_string(index=False))

# Find best model for each metric
print("\n\nBest Model by Metric:")
print("-"*40)
for col in ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC-AUC']:
    best_idx = comparison_df[col].idxmax()
    best_model = comparison_df.loc[best_idx, 'Model']
    best_value = comparison_df.loc[best_idx, col]
    print(f"{col}: {best_model} ({best_value*100:.2f}%)")

In [ ]:
# Visualize model comparison
fig, ax = plt.subplots(figsize=(12, 6))

metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC-AUC']
x = np.arange(len(metrics))
width = 0.2
multiplier = 0

colors = ['#3498db', '#2ecc71', '#e74c3c', '#9b59b6']

for idx, (model_name, results) in enumerate(training_results.items()):
    values = [results['accuracy'], results['precision'], results['recall'], 
              results['f1'], results['roc_auc']]
    offset = width * multiplier
    bars = ax.bar(x + offset, values, width, label=model_name, color=colors[idx])
    multiplier += 1

ax.set_ylabel('Score', fontsize=12)
ax.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels(metrics, fontsize=11)
ax.legend(loc='lower right', fontsize=10)
ax.set_ylim(0, 1.1)
ax.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 11. Confusion Matrix Analysis

### What is a Confusion Matrix?

A confusion matrix is a table that visualizes the performance of a classification model by showing the counts of:
- **True Positives (TP)**: Correctly predicted frauds
- **True Negatives (TN)**: Correctly predicted legitimate transactions
- **False Positives (FP)**: Legitimate transactions incorrectly flagged as fraud
- **False Negatives (FN)**: Frauds that were not detected

```
                    Predicted
                 Negative  Positive
Actual Negative    TN        FP
Actual Positive    FN        TP
```

### Business Impact:
- **FP (False Alarms)**: Customer inconvenience, declined legitimate purchases
- **FN (Missed Frauds)**: Financial losses, customer trust damage

In [ ]:
# Plot confusion matrices for all models
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, (model_name, results) in enumerate(training_results.items()):
    cm = confusion_matrix(y_test, results['y_pred'])
    
    # Calculate percentages
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    
    # Create annotations with both count and percentage
    annotations = np.array([[f'{cm[i,j]:,}\n({cm_percent[i,j]:.1f}%)' 
                            for j in range(2)] for i in range(2)])
    
    sns.heatmap(cm, annot=annotations, fmt='', cmap='Blues', ax=axes[idx],
                xticklabels=['Legitimate', 'Fraud'],
                yticklabels=['Legitimate', 'Fraud'],
                cbar_kws={'label': 'Count'})
    
    axes[idx].set_title(f'{model_name}\nAccuracy: {results["accuracy"]*100:.2f}%', 
                        fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Actual', fontsize=11)
    axes[idx].set_xlabel('Predicted', fontsize=11)

plt.suptitle('Confusion Matrices - All Models', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('confusion_matrices.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Detailed confusion matrix analysis
print("="*60)
print("DETAILED CONFUSION MATRIX ANALYSIS")
print("="*60)

for model_name, results in training_results.items():
    cm = confusion_matrix(y_test, results['y_pred'])
    tn, fp, fn, tp = cm.ravel()
    
    print(f"\n{model_name}:")
    print("-"*40)
    print(f"  True Negatives (TN):  {tn:,} - Correctly identified legitimate")
    print(f"  False Positives (FP): {fp:,} - Legitimate flagged as fraud (false alarm)")
    print(f"  False Negatives (FN): {fn:,} - Fraud not detected (MISSED!)")
    print(f"  True Positives (TP):  {tp:,} - Correctly identified fraud")
    print(f"\n  Detection Rate: {tp/(tp+fn)*100:.2f}% of frauds caught")
    print(f"  False Alarm Rate: {fp/(fp+tn)*100:.2f}% of legitimate flagged")

## 12. ROC Curve and AUC Analysis

### What is ROC-AUC?

**ROC (Receiver Operating Characteristic)** curve plots:
- **True Positive Rate (Sensitivity/Recall)** vs **False Positive Rate (1 - Specificity)**
- At various classification thresholds

**AUC (Area Under Curve)**:
- Ranges from 0 to 1
- 0.5 = Random guessing (diagonal line)
- 1.0 = Perfect classifier
- Higher is better

### Interpretation:
- **AUC > 0.9**: Excellent
- **AUC 0.8-0.9**: Good
- **AUC 0.7-0.8**: Fair
- **AUC < 0.7**: Poor

In [ ]:
# Plot ROC curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = ['#3498db', '#2ecc71', '#e74c3c', '#9b59b6']

# ROC Curve
for idx, (model_name, results) in enumerate(training_results.items()):
    fpr, tpr, _ = roc_curve(y_test, results['y_pred_proba'])
    auc = roc_auc_score(y_test, results['y_pred_proba'])
    axes[0].plot(fpr, tpr, color=colors[idx], lw=2, 
                 label=f'{model_name} (AUC = {auc:.4f})')

axes[0].plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier')
axes[0].set_xlim([0.0, 1.0])
axes[0].set_ylim([0.0, 1.05])
axes[0].set_xlabel('False Positive Rate', fontsize=12)
axes[0].set_ylabel('True Positive Rate', fontsize=12)
axes[0].set_title('ROC Curves', fontsize=14, fontweight='bold')
axes[0].legend(loc='lower right', fontsize=10)
axes[0].grid(True, alpha=0.3)

# Precision-Recall Curve
for idx, (model_name, results) in enumerate(training_results.items()):
    precision, recall, _ = precision_recall_curve(y_test, results['y_pred_proba'])
    ap = average_precision_score(y_test, results['y_pred_proba'])
    axes[1].plot(recall, precision, color=colors[idx], lw=2, 
                 label=f'{model_name} (AP = {ap:.4f})')

axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('Recall', fontsize=12)
axes[1].set_ylabel('Precision', fontsize=12)
axes[1].set_title('Precision-Recall Curves', fontsize=14, fontweight='bold')
axes[1].legend(loc='lower left', fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('roc_pr_curves.png', dpi=150, bbox_inches='tight')
plt.show()

## 13. Cross-Validation

### Why Cross-Validation?

A single train-test split might give results that depend on how the data was split. **Cross-validation** provides a more robust estimate by:

1. Splitting data into K folds
2. Training K times, each time using a different fold as validation
3. Averaging results across all folds

### Stratified K-Fold:
We use **Stratified** K-Fold to maintain class proportions in each fold, which is especially important for classification problems.

In [ ]:
print("="*60)
print("CROSS-VALIDATION (5-Fold Stratified)")
print("="*60)

# Perform 5-fold stratified cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

cv_results = {}

print("\nPerforming cross-validation for each model...")
print("(This may take a moment)\n")

for name, model in models.items():
    print(f"Cross-validating {name}...")
    
    # Calculate CV scores for multiple metrics
    accuracy_scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
    precision_scores = cross_val_score(model, X, y, cv=cv, scoring='precision')
    recall_scores = cross_val_score(model, X, y, cv=cv, scoring='recall')
    f1_scores = cross_val_score(model, X, y, cv=cv, scoring='f1')
    roc_auc_scores = cross_val_score(model, X, y, cv=cv, scoring='roc_auc')
    
    cv_results[name] = {
        'Accuracy': (accuracy_scores.mean(), accuracy_scores.std()),
        'Precision': (precision_scores.mean(), precision_scores.std()),
        'Recall': (recall_scores.mean(), recall_scores.std()),
        'F1 Score': (f1_scores.mean(), f1_scores.std()),
        'ROC-AUC': (roc_auc_scores.mean(), roc_auc_scores.std())
    }

print("\n Cross-validation complete!")

In [ ]:
# Display CV results
print("\nCross-Validation Results (Mean ± Std):")
print("="*80)

for name, results in cv_results.items():
    print(f"\n{name}:")
    print("-"*50)
    for metric, (mean, std) in results.items():
        print(f"  {metric:12s}: {mean*100:.2f}% (± {std*100:.2f}%)")

print("\n" + "="*80)
print("\nInterpretation:")
print("-"*50)
print(" Low standard deviation indicates consistent performance across folds")
print(" High standard deviation suggests the model is sensitive to data splits")

## 14. Feature Importance Analysis

Understanding which features are most important helps us:
1. Interpret the model's decisions
2. Potentially reduce features for simpler models
3. Gain business insights about fraud patterns

In [ ]:
# Get feature importances from tree-based models
print("="*60)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*60)

# Random Forest feature importance
rf_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': trained_models['Random Forest'].feature_importances_
}).sort_values('Importance', ascending=False)

# XGBoost feature importance
xgb_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': trained_models['XGBoost'].feature_importances_
}).sort_values('Importance', ascending=False)

print("\nTop 10 Features - Random Forest:")
print("-"*40)
print(rf_importance.head(10).to_string(index=False))

print("\n\nTop 10 Features - XGBoost:")
print("-"*40)
print(xgb_importance.head(10).to_string(index=False))

In [ ]:
# Visualize feature importance
fig, axes = plt.subplots(1, 2, figsize=(14, 8))

# Random Forest
top_rf = rf_importance.head(15)
axes[0].barh(range(len(top_rf)), top_rf['Importance'].values, color='#3498db')
axes[0].set_yticks(range(len(top_rf)))
axes[0].set_yticklabels(top_rf['Feature'].values)
axes[0].invert_yaxis()
axes[0].set_xlabel('Importance', fontsize=12)
axes[0].set_title('Random Forest - Feature Importance', fontsize=14, fontweight='bold')

# XGBoost
top_xgb = xgb_importance.head(15)
axes[1].barh(range(len(top_xgb)), top_xgb['Importance'].values, color='#9b59b6')
axes[1].set_yticks(range(len(top_xgb)))
axes[1].set_yticklabels(top_xgb['Feature'].values)
axes[1].invert_yaxis()
axes[1].set_xlabel('Importance', fontsize=12)
axes[1].set_title('XGBoost - Feature Importance', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

## 15. Unsupervised Anomaly Detection

For comparison, let's also try **Isolation Forest**, an unsupervised anomaly detection algorithm.

### How Isolation Forest Works:

1. Builds random trees by randomly selecting features and split values
2. Anomalies are isolated earlier (fewer splits needed) because they are different from normal points
3. Normal points require more splits to isolate

This approach is useful when labeled data is scarce or unavailable.

In [ ]:
print("="*60)
print("UNSUPERVISED ANOMALY DETECTION - Isolation Forest")
print("="*60)

# Train Isolation Forest
# contamination is set based on our knowledge of fraud ratio
fraud_ratio = (y == 1).sum() / len(y)
print(f"\nSetting contamination parameter to: {fraud_ratio:.4f}")
print("(This tells the model the expected proportion of anomalies)")

iso_forest = IsolationForest(
    n_estimators=100,
    contamination=fraud_ratio,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

# Fit and predict
iso_forest.fit(X_train)
y_pred_iso = iso_forest.predict(X_test)

# Convert predictions: Isolation Forest returns -1 for anomalies, 1 for normal
y_pred_iso = np.where(y_pred_iso == -1, 1, 0)

# Evaluate
print("\nIsolation Forest Results:")
print("-"*40)
print(f"Accuracy:  {accuracy_score(y_test, y_pred_iso)*100:.2f}%")
print(f"Precision: {precision_score(y_test, y_pred_iso)*100:.2f}%")
print(f"Recall:    {recall_score(y_test, y_pred_iso)*100:.2f}%")
print(f"F1 Score:  {f1_score(y_test, y_pred_iso)*100:.2f}%")

print("\nNote: Isolation Forest is unsupervised, so it doesn't use labels during training.")
print("Performance is lower than supervised methods but useful when labels are unavailable.")

In [ ]:
# Confusion Matrix for Isolation Forest
print("\n" + "="*60)
print("ISOLATION FOREST - CONFUSION MATRIX")
print("="*60)

cm_iso = confusion_matrix(y_test, y_pred_iso)
tn_iso, fp_iso, fn_iso, tp_iso = cm_iso.ravel()

# Calculate percentages
cm_iso_percent = cm_iso.astype('float') / cm_iso.sum(axis=1)[:, np.newaxis] * 100

# Create figure
plt.figure(figsize=(8, 6))

# Create annotations with both count and percentage
annotations_iso = np.array([[f'TN\n{cm_iso[0,0]:,}\n({cm_iso_percent[0,0]:.1f}%)', 
                             f'FP\n{cm_iso[0,1]:,}\n({cm_iso_percent[0,1]:.1f}%)'],
                            [f'FN\n{cm_iso[1,0]:,}\n({cm_iso_percent[1,0]:.1f}%)', 
                             f'TP\n{cm_iso[1,1]:,}\n({cm_iso_percent[1,1]:.1f}%)']])

sns.heatmap(cm_iso, annot=annotations_iso, fmt='', cmap='Oranges',
            xticklabels=['Predicted\nLegitimate', 'Predicted\nFraudulent'],
            yticklabels=['Actual\nLegitimate', 'Actual\nFraudulent'],
            cbar_kws={'label': 'Count'})

plt.title('Confusion Matrix - Isolation Forest (Unsupervised)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('isolation_forest_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Detailed analysis
print("\nIsolation Forest - Confusion Matrix Analysis:")
print("-"*50)
print(f"  True Negatives (TN):  {tn_iso:,} - Correctly identified legitimate")
print(f"  False Positives (FP): {fp_iso:,} - Legitimate flagged as fraud")
print(f"  False Negatives (FN): {fn_iso:,} - Fraud not detected (MISSED!)")
print(f"  True Positives (TP):  {tp_iso:,} - Correctly identified fraud")
print(f"\n  Detection Rate: {tp_iso/(tp_iso+fn_iso)*100:.2f}% of frauds caught")
print(f"  False Alarm Rate: {fp_iso/(fp_iso+tn_iso)*100:.2f}% of legitimate flagged")

print("\n" + "-"*50)
print("Comparison Note:")
print("Isolation Forest is unsupervised - it detects anomalies based on")
print("how easily points can be 'isolated' in random trees, without using class labels.")
print("This makes it valuable when labeled fraud data is scarce or unavailable.")

## 16. Best Model Selection and Final Classification Report

In [ ]:
print("="*60)
print("BEST MODEL SELECTION")
print("="*60)

# Find best model based on F1 Score (balanced metric for fraud detection)
best_model_name = max(training_results, key=lambda x: training_results[x]['f1'])
best_model = trained_models[best_model_name]
best_results = training_results[best_model_name]

print(f"\nBest Model: {best_model_name}")
print(f"   Selection Criterion: Highest F1 Score")
print(f"   (F1 Score balances precision and recall - ideal for fraud detection)")

print("\n\nFinal Performance Metrics:")
print("-"*40)
print(f"  Accuracy:  {best_results['accuracy']*100:.2f}%")
print(f"  Precision: {best_results['precision']*100:.2f}%")
print(f"  Recall:    {best_results['recall']*100:.2f}%")
print(f"  F1 Score:  {best_results['f1']*100:.2f}%")
print(f"  ROC-AUC:   {best_results['roc_auc']*100:.2f}%")

In [ ]:
# Detailed classification report for best model
print("\n" + "="*60)
print(f"DETAILED CLASSIFICATION REPORT - {best_model_name}")
print("="*60)

print("\n" + classification_report(y_test, best_results['y_pred'], 
                                    target_names=['Legitimate', 'Fraudulent']))

print("\nMetric Definitions:")
print("-"*40)
print(" Precision: Of all predicted frauds, what % are actually frauds?")
print(" Recall: Of all actual frauds, what % did we catch?")
print(" F1-Score: Harmonic mean of precision and recall")
print(" Support: Number of actual occurrences of each class")

In [ ]:
# Final confusion matrix for best model
cm = confusion_matrix(y_test, best_results['y_pred'])
tn, fp, fn, tp = cm.ravel()

plt.figure(figsize=(8, 6))

# Create annotations
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
annotations = np.array([[f'TN\n{cm[0,0]:,}\n({cm_percent[0,0]:.1f}%)', 
                         f'FP\n{cm[0,1]:,}\n({cm_percent[0,1]:.1f}%)'],
                        [f'FN\n{cm[1,0]:,}\n({cm_percent[1,0]:.1f}%)', 
                         f'TP\n{cm[1,1]:,}\n({cm_percent[1,1]:.1f}%)']])

sns.heatmap(cm, annot=annotations, fmt='', cmap='Blues',
            xticklabels=['Predicted\nLegitimate', 'Predicted\nFraudulent'],
            yticklabels=['Actual\nLegitimate', 'Actual\nFraudulent'],
            cbar_kws={'label': 'Count'})

plt.title(f'Final Confusion Matrix - {best_model_name}', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('final_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nBusiness Impact Analysis:")
print("-"*40)
print(f" Frauds Detected (TP): {tp:,} out of {tp+fn:,} ({tp/(tp+fn)*100:.2f}%)")
print(f" Frauds Missed (FN): {fn:,} - These would result in financial losses")
print(f" False Alarms (FP): {fp:,} - Legitimate transactions incorrectly blocked")
print(f" Correct Approvals (TN): {tn:,} - Legitimate transactions correctly approved")

## 17. Summary and Conclusions

In [ ]:
print("="*70)
print("                    PROJECT SUMMARY AND CONCLUSIONS")
print("="*70)

print("""
DATASET CHARACTERISTICS:
──────────────────────────────────────────────────────────────────────""")
print(f"    Total Transactions: {len(df):,}")
print(f"    Features: 29 (V1-V28 + Amount)")
print(f"    Class Distribution: {'Balanced' if 0.8 <= class_percentages[1]/class_percentages[0] <= 1.2 else 'Imbalanced'}")
print(f"    Legitimate: {class_counts[0]:,} ({class_percentages[0]:.2f}%)")
print(f"    Fraudulent: {class_counts[1]:,} ({class_percentages[1]:.2f}%)")

print("""
PREPROCESSING STEPS:
──────────────────────────────────────────────────────────────────────""")
print("   1. Removed non-predictive 'id' column")
print("   2. Scaled 'Amount' feature using RobustScaler")
print("   3. Performed 80/20 stratified train-test split")

print("""
MODELS EVALUATED:
──────────────────────────────────────────────────────────────────────""")
print("    Logistic Regression (baseline)")
print("    Decision Tree")
print("    Random Forest")
print("    XGBoost")
print("    Isolation Forest (unsupervised, for comparison)")

print("""
MODEL PERFORMANCE SUMMARY:
──────────────────────────────────────────────────────────────────────""")
print(comparison_display.to_string(index=False))

print(f"""
🏆 BEST MODEL: {best_model_name}
──────────────────────────────────────────────────────────────────────
   Selection Criterion: Highest F1 Score
   
   Final Metrics:
    Accuracy:  {best_results['accuracy']*100:.2f}%
    Precision: {best_results['precision']*100:.2f}%
    Recall:    {best_results['recall']*100:.2f}%
    F1 Score:  {best_results['f1']*100:.2f}%
    ROC-AUC:   {best_results['roc_auc']*100:.2f}%
""")

print("""
KEY INSIGHTS:
──────────────────────────────────────────────────────────────────────""")
print("   1. TO BE DONE...")
print("   2. TO BE DONE...")
print("   3. TO BE DONE...")

print("\n" + "="*70)
print("                         END OF ANALYSIS")
print("="*70)

## 18. Save the Best Model (Optional)

In [ ]:
# Save the best model for future use
import joblib

# Save model
model_filename = f'fraud_detection_model_{best_model_name.lower().replace(" ", "_")}.joblib'
joblib.dump(best_model, model_filename)
print(f" Model saved as '{model_filename}'")

# Save scaler
joblib.dump(scaler, 'amount_scaler.joblib')
print(" Scaler saved as 'amount_scaler.joblib'")

print("\n--- How to load and use the model ---")
print("""
# Load model and scaler
model = joblib.load('fraud_detection_model_xxx.joblib')
scaler = joblib.load('amount_scaler.joblib')

# Preprocess new data
new_data['Amount'] = scaler.transform(new_data[['Amount']])

# Make predictions
predictions = model.predict(new_data)
probabilities = model.predict_proba(new_data)[:, 1]
""")